In [1]:
import os
SNAPPY_notebook_path = os.path.join(os.path.abspath(""), "bench.ipynb")

In [2]:
import time
SNAPPY_start_time = time.perf_counter_ns()

In [3]:
%load_ext autotime

time: 38.1 µs (started: 2024-05-07 14:42:48 -04:00)


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import os

import pandas as pd
# import seaborn as sns  # data visualization
# import matplotlib.pyplot as plt


# Input data files are available in the read-only "input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(os.path.abspath('') + '/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/sam/dias-benchmarks/dias-benchmarks/notebooks/ampiiere/animal-crossing-villager-popularity-analysis/input/animal-crossing-new-horizons-nookplaza-dataset/villagers.csv
/home/sam/dias-benchmarks/dias-benchmarks/notebooks/ampiiere/animal-crossing-villager-popularity-analysis/input/acnh-villager-popularity/acnh_villager_data.csv
time: 225 ms (started: 2024-05-07 14:42:48 -04:00)


# Introduction
The goal of this project is to analyse the relationship between animal crossing new horizon villager popularity amongst the player base and certain villager attributes. 

We will be analysing the Gender,Personality, Species, and Style of a villager. 

# Data Initilization and Cleaning

In [5]:
vlgr_df = pd.read_csv(os.path.abspath('') + "/input/animal-crossing-new-horizons-nookplaza-dataset/villagers.csv")
popul_df = pd.read_csv(os.path.abspath('') + "/input/acnh-villager-popularity/acnh_villager_data.csv")

time: 7.68 ms (started: 2024-05-07 14:42:49 -04:00)


# -- STEFANOS -- Replicate Data

In [6]:
factor = 400
if "IREWR_LESS_REPLICATION" in os.environ and os.environ["IREWR_LESS_REPLICATION"] == "True":
    factor = factor//3
popul_df = pd.concat([popul_df]*factor, ignore_index=True)
vlgr_df = pd.concat([vlgr_df]*factor, ignore_index=True)
# print(popul_df.info())
# vlgr_df.info()

time: 17.6 ms (started: 2024-05-07 14:42:49 -04:00)


In [7]:
vlgr_df.head()

,Name,Species,Gender,Personality,Hobby,Birthday,Catchphrase,Favorite Song,Style 1,Style 2,Color 1,Color 2,Wallpaper,Flooring,Furniture List,Filename,Unique Entry ID
0,Admiral,Bird,Male,Cranky,Nature,27-Jan,aye aye,Steep Hill,Cool,Cool,Black,Blue,dirt-clod wall,tatami,717;1849;7047;2736;787;5970;3449;3622;3802;410...,brd06,B3RyfNEqwGmcccRC3
1,Agent S,Squirrel,Female,Peppy,Fitness,2-Jul,sidekick,Go K.K. Rider,Active,Simple,Blue,Black,concrete wall,colorful tile flooring,7845;7150;3468;4080;290;3971;3449;1708;4756;25...,squ05,SGMdki6dzpDZyXAw5
2,Agnes,Pig,Female,Big Sister,Play,21-Apr,snuffle,K.K. House,Simple,Elegant,Pink,White,gray molded-panel wall,arabesque flooring,4129;7236;7235;7802;896;3428;4027;7325;3958;71...,pig17,jzWCiDPm9MqtCfecP
3,Al,Gorilla,Male,Lazy,Fitness,18-Oct,ayyyeee,Go K.K. Rider,Active,Active,Red,White,concrete wall,green rubber flooring,1452;4078;4013;833;4116;3697;7845;3307;3946;39...,gor08,LBifxETQJGEaLhBjC
4,Alfonso,Alligator,Male,Lazy,Play,9-Jun,it'sa me,Forest Life,Simple,Simple,Red,Blue,yellow playroom wall,green honeycomb tile,4763;3205;3701;1557;3623;85;3208;3584;4761;121...,crd00,REpd8KxB8p9aGBRSE


time: 9 ms (started: 2024-05-07 14:42:49 -04:00)


In [8]:
popul_df.head()

,tier,rank,name
0,1,1,Raymond
1,1,2,Marshal
2,1,3,Shino
3,1,4,Sherb
4,1,5,Sasha


time: 2.72 ms (started: 2024-05-07 14:42:49 -04:00)


### 1. Checking for null 

In [9]:
# vlgr_df.info()

time: 102 µs (started: 2024-05-07 14:42:49 -04:00)


In [10]:
# popul_df.info()

time: 201 µs (started: 2024-05-07 14:42:49 -04:00)


### 2. Checking for mismatched names

In [11]:
# There are some missing/non-matching names 
vlgr_df["Name"].isin(popul_df['name']).sum()

154400

time: 5.19 ms (started: 2024-05-07 14:42:49 -04:00)


In [12]:
# vlgr_df does not have these names...
mismatch_names = popul_df["name"][popul_df["name"].isin(vlgr_df["Name"]) == False]
mismatch_names

2                  Shino
4                  Sasha
5                   Ione
25            Cephalobot
57                Étoile
               ...      
165087    Crackle(Spork)
165110               Ace
165115              Toby
165128             Frett
165186             Rilla
Name: name, Length: 10800, dtype: object

time: 5.91 ms (started: 2024-05-07 14:42:49 -04:00)


In [13]:
# %%time
# Data set is small enough to pick out the same names
# Correcting names in popul_df to match vlgr_df
popul_df['name'] = popul_df['name'].replace(['OHare'],"O\'Hare")
popul_df['name'] = popul_df['name'].replace(['Buck(Brows)'],"Buck")
popul_df['name'] = popul_df['name'].replace(['Renee'],"Renée")
popul_df['name'] = popul_df['name'].replace(['WartJr'],"Wart Jr.")
popul_df['name'] = popul_df['name'].replace(['Crackle(Spork)'],"Spork")

time: 34.3 ms (started: 2024-05-07 14:42:49 -04:00)


In [14]:
# Checking if All names match
vlgr_df["Name"].isin(popul_df['name']).sum()

156400

time: 5.05 ms (started: 2024-05-07 14:42:49 -04:00)


In [15]:
# drop villagers that are in popul_df but not in vlgr_df
popul_df = popul_df.drop(popul_df[popul_df["name"].isin(vlgr_df["Name"]) == False].index)

time: 7.1 ms (started: 2024-05-07 14:42:49 -04:00)


### 3. Merging the two Dataframes

In [16]:
# Now that both df have same length, we can set index as names and combine the 2 dfs
popul_df.set_index('name', drop=True, inplace=True)
vlgr_df.set_index('Name', drop=True, inplace=True)

time: 769 µs (started: 2024-05-07 14:42:49 -04:00)


In [17]:
combined_df = popul_df.merge(vlgr_df, left_index=True, right_index=True)

time: 31.3 s (started: 2024-05-07 14:42:49 -04:00)


In [18]:
# drop irrelevent columns
combined_df.drop(columns=['Furniture List', 'Filename', 'Unique Entry ID', "Wallpaper", "Flooring", "Birthday", "Favorite Song"], inplace=True)

time: 2.48 s (started: 2024-05-07 14:43:21 -04:00)


#### Adding a new row named overall_ranking so we may know a villager's general ranking outside of their tier

In [19]:
combined_df.sort_values(['tier', 'rank'], inplace=True)
combined_df['overall_ranking'] = np.arange(1, len(combined_df)+1)
combined_df.insert(2, 'overall_ranking', combined_df.pop('overall_ranking'))

time: 8.35 s (started: 2024-05-07 14:43:23 -04:00)


#### Setting Baseline overall ranking mean to compare against

In [20]:
overall_mean = combined_df.overall_ranking.mean()
print(f'The overall_mean is {overall_mean}, this would serve as a baseline for to compare against popularity performance of our features.')

The overall_mean is 31280000.5, this would serve as a baseline for to compare against popularity performance of our features.
time: 46.8 ms (started: 2024-05-07 14:43:32 -04:00)


In [21]:
combined_df.columns

Index(['tier', 'rank', 'overall_ranking', 'Species', 'Gender', 'Personality',
       'Hobby', 'Catchphrase', 'Style 1', 'Style 2', 'Color 1', 'Color 2'],
      dtype='object')

time: 23.1 ms (started: 2024-05-07 14:43:32 -04:00)


# Exploratory Data Analysis
As a preface, a higher overall_ranking would mean performing worse on the popularity rankings.
### 1. Gender

In [22]:
combined_df['Gender'].value_counts()

Gender
Male      32640000
Female    29920000
Name: count, dtype: int64

time: 1.46 s (started: 2024-05-07 14:43:32 -04:00)


In [23]:
## -- STEFANOS: Disable plotting
# combined_df.groupby('tier').Gender.value_counts().plot.barh()
## -- STEFANOS-DISABLE-FOR-MODIN: Actually, it seems we can't even run the last part with Modin, so
## we only leave the groupby.
# combined_df.groupby('tier').Gender.value_counts()
combined_df.groupby('tier')

time: 10.2 ms (started: 2024-05-07 14:43:33 -04:00)


For gender, there seems to be a disproporationate amount of male villagers in the lowest tier(6th tier) than female villagers, compared to other tiers. Discounting Tier 6, The number of male and female villagers are fairly even, with Male villagers having a slight lead in all tiers(except tier 6).

In [24]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(5, 5))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Gender", y='overall_ranking', data=combined_df)

time: 372 µs (started: 2024-05-07 14:43:33 -04:00)


Female villagers generally perform better than Male villagers in terms of overall ranking. 

In [25]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Gender", aggfunc='count')

Gender,Female,Male
tier,,
1,960000,960000
2,2240000,1600000
3,2880000,1760000
4,4800000,4000000
5,9280000,8800000
6,9760000,15520000


time: 4.34 s (started: 2024-05-07 14:43:33 -04:00)


### 2. Species

In [26]:
# combined_df.info()

time: 128 µs (started: 2024-05-07 14:43:38 -04:00)


In [27]:
combined_df.groupby('Species').mean(numeric_only=True)

,tier,rank,overall_ranking
Species,,,
Alligator,5.285714,55.428571,3.541714e+07
Anteater,5.571429,51.000000,3.731429e+07
Bear,5.400000,67.200000,3.853333e+07
Bird,5.769231,73.538462,4.432615e+07
Bull,5.666667,75.000000,4.269333e+07
Cat,3.434783,28.826087,1.504348e+07
Chicken,5.555556,74.888889,4.072000e+07
Cow,5.250000,59.500000,3.280000e+07
Cub,4.062500,38.437500,2.283000e+07


time: 2.63 s (started: 2024-05-07 14:43:38 -04:00)


In [28]:
# creating value counts dataframe for each species type
species_ranking = combined_df.groupby('Species').mean(numeric_only=True)['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')
species_ranking

,Species,overall_ranking
25,Octopus,2.693334e+06
9,Deer,8.288000e+06
34,Wolf,1.154182e+07
5,Cat,1.504348e+07
21,Koala,2.137778e+07
10,Dog,2.228000e+07
8,Cub,2.283000e+07
17,Hamster,2.348000e+07
32,Squirrel,2.369778e+07
30,Rhino,2.477333e+07


time: 2.62 s (started: 2024-05-07 14:43:40 -04:00)


In [29]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(30,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Species', y="overall_ranking", data=species_ranking,label='mean overall-ranking', s=300)

time: 221 µs (started: 2024-05-07 14:43:43 -04:00)


Octopus, deer, wolves, cats and Koalas are most likely to be popular; while Kangaroos, Hippos, Mouse Pigs and Gorillas are the least likely to be popular. 

In [30]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(30, 10))
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x="Species", y='overall_ranking', hue='tier', s=100, data=combined_df)

time: 168 µs (started: 2024-05-07 14:43:43 -04:00)


Although Octopuses seem to be ranking highly in part due to the low amount of Octopuses amongst the villagers. 
Interesting trend can be observed, there exists a ranking cap for low ranking speices, for example, none of the Gorilla villagers have a ranking lower than 200, it is heavily skewed, and not normally distributed.  Indicating a clear non-preference for certain species by the playerbase. 

### 3. Personality

In [31]:
combined_df.Personality.value_counts()

Personality
Lazy          9600000
Normal        9440000
Snooty        8800000
Jock          8800000
Cranky        8800000
Peppy         7840000
Smug          5440000
Big Sister    3840000
Name: count, dtype: int64

time: 1.44 s (started: 2024-05-07 14:43:43 -04:00)


In [32]:
# creating value counts dataframe for each personality type
personality_ranking = combined_df.groupby('Personality').mean(numeric_only=True)['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')

time: 2.6 s (started: 2024-05-07 14:43:44 -04:00)


In [33]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(20,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Personality', y="overall_ranking", data=personality_ranking,label='mean personality ranking', s=300)

time: 199 µs (started: 2024-05-07 14:43:47 -04:00)


The playerbase seems to have a preference for Big sister, Normal, Peppy and sometimes Lazy type villagers.
While they dislike Cranky, Jock and Snooty villagers. 

In [34]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(10, 10))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Personality", y='overall_ranking', data=combined_df)

time: 235 µs (started: 2024-05-07 14:43:47 -04:00)


There seems to be a clear preference for Big Sister, Peppy and Normal Personality villagers, they have means below overall mean. Rankings are fairly normally distributed except for Smug villagers. On the other hand, Cranky and Snooty both have a mean clearly above the overall mean.

In [35]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Personality", aggfunc='count')

Personality,Big Sister,Cranky,Jock,Lazy,Normal,Peppy,Smug,Snooty
tier,,,,,,,,
1,NaN,NaN,NaN,640000.0,640000.0,NaN,320000.0,320000.0
2,160000.0,320000.0,320000.0,640000.0,960000.0,960000.0,320000.0,160000.0
3,800000.0,640000.0,480000.0,320000.0,1280000.0,640000.0,320000.0,160000.0
4,960000.0,800000.0,800000.0,1440000.0,1280000.0,1600000.0,960000.0,960000.0
5,1280000.0,2080000.0,2880000.0,3040000.0,3040000.0,2560000.0,800000.0,2400000.0
6,640000.0,4960000.0,4320000.0,3520000.0,2240000.0,2080000.0,2720000.0,4800000.0


time: 4.27 s (started: 2024-05-07 14:43:47 -04:00)


### 4. Style

In [36]:
# generating value counts dataframe for each style type
style_ranking1 = combined_df.groupby('Style 1').mean(numeric_only=True)['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')
style_ranking2 = combined_df.groupby('Style 2').mean(numeric_only=True)['overall_ranking'].to_frame().reset_index().sort_values('overall_ranking')

time: 5.21 s (started: 2024-05-07 14:43:51 -04:00)


In [37]:
# combining the 2 style columns and finding a mean
style_ranking = style_ranking1.copy()
style_series = (style_ranking1['overall_ranking'] + style_ranking2['overall_ranking'])/2
style_ranking["overall_ranking"] = style_series

time: 8.15 ms (started: 2024-05-07 14:43:56 -04:00)


In [38]:
style_ranking

,Style 1,overall_ranking
2,Cute,2.200213e+07
5,Simple,2.976031e+07
3,Elegant,3.350778e+07
1,Cool,3.460626e+07
4,Gorgeous,3.504125e+07
0,Active,3.610720e+07


time: 2.81 ms (started: 2024-05-07 14:43:56 -04:00)


In [39]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(20,5))
# sns.set(font_scale=1.4)
# plt.xticks(rotation=45)
# plt.axhline(overall_mean, color='r')
# sns.scatterplot(x='Style 1', y="overall_ranking", data=style_ranking, s=300)

time: 184 µs (started: 2024-05-07 14:43:56 -04:00)


A very clear preference for Cute styled villagers. Simple Styled Villagers have a ranking mean just about equal to the overall mean, while other style villagers have a slightly above overall mean mean. 

In [40]:
## -- STEFANOS: Disable plotting
# plt.figure(figsize=(7, 7))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Style 1", y='overall_ranking', data=combined_df)
# plt.title('Style 1')
# plt.figure(figsize=(7, 7))
# plt.axhline(overall_mean, color='r')
# sns.boxplot(x="Style 2", y='overall_ranking', data=combined_df)
# plt.title('Style 2')

time: 327 µs (started: 2024-05-07 14:43:56 -04:00)


The clear preference is Cute style dressing, in both Style columns. In particular, in Style 2 column Cute Styled Villagers have a higher concetration in lower rankings. Other styles seem to have a fairly normally distributed ranking, with the exception of Active Style Villagers in Style 1, right skewed, but the ranking mean is significantly above the overall ranking mean.  

In [41]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Style 1", aggfunc='count')

Style 1,Active,Cool,Cute,Elegant,Gorgeous,Simple
tier,,,,,,
1,NaN,NaN,480000.0,320000.0,160000.0,960000.0
2,320000.0,320000.0,1280000.0,160000.0,320000.0,1440000.0
3,480000.0,1120000.0,1280000.0,800000.0,NaN,960000.0
4,320000.0,1760000.0,1760000.0,1120000.0,960000.0,2880000.0
5,2400000.0,2720000.0,2240000.0,2720000.0,1760000.0,6240000.0
6,4480000.0,4960000.0,3040000.0,3520000.0,2880000.0,6400000.0


time: 4.19 s (started: 2024-05-07 14:43:56 -04:00)


In [42]:
pd.pivot_table(combined_df, index = 'tier', values = 'Catchphrase', columns="Style 2", aggfunc='count')

Style 2,Active,Cool,Cute,Elegant,Gorgeous,Simple
tier,,,,,,
1,NaN,320000.0,1120000.0,160000.0,NaN,320000.0
2,320000.0,320000.0,960000.0,320000.0,160000.0,1760000.0
3,320000.0,480000.0,1600000.0,480000.0,320000.0,1440000.0
4,1280000.0,1120000.0,2080000.0,1280000.0,1280000.0,1760000.0
5,2080000.0,2400000.0,2880000.0,2400000.0,2880000.0,5440000.0
6,4000000.0,4800000.0,1280000.0,4320000.0,4000000.0,6880000.0


time: 4.19 s (started: 2024-05-07 14:44:01 -04:00)


# Conclusion
We may come to the conclusion, that the following attributes contribute to a villager's popularity:
- Gender: Despite Female Villagers having in general better popularity, this is likely due to the overwheling prescence of male villagers in the lowest tier. Other than the lowest tier, Male villagers in general perform slightly better.
- Species: Octopus, Wolf, Deer and Cat villagers perform the best. 
- Personality: Big Sister, Normal and Peppy villagers are in general the most popular. 
- Style: Cute Style villagers are very clearly the most popular

In [43]:
SNAPPY_end_time = time.perf_counter_ns()
print("Total elapsed time:", (SNAPPY_end_time - SNAPPY_start_time) / (10 ** 9))

Total elapsed time: 76.466339666
time: 390 µs (started: 2024-05-07 14:44:05 -04:00)
